In [1]:
from vpython import *
import random
import numpy as np

<IPython.core.display.Javascript object>

## Final Project: Asynchronous Wildfire Simulation

In [2]:
dir_x = [1, -1, 0, 0, 1, -1, 1, -1]
dir_y = [0, 0, -1, 1, 1, -1, -1, 1]

burn_x = [1, -1]
burn_y = [-1, -1]

# Update the next state of given cell (for async)
def update_forest_arr_async(n, cell):
    i, j = cell
    cnt = 0
    state = forest_arr[i][j].get_state()
            
    for k in range(8):
        # check out-of-bound case
        nx = i + dir_x[k]
        ny = j + dir_y[k]
        if nx < 0 or ny < 0 or nx >= n or ny >= n:
            continue
        if forest_arr[nx][ny].get_state() == 'fire' or forest_arr[nx][ny].get_state() == 'burned':
            cnt += 1
                    
    # if current cell is a tree & fire surrounding
    if state == 'tree':
        if cnt >= 1:
            forest_arr[i][j].set_state('fire')
    # if current cell is a fire
    elif state == 'fire':
        fire_list.remove((i,j))
        forest_arr[i][j].set_state('burnt')
        if cnt >= 6:
            for k in range(2):
                nx = i + burn_x[k]
                ny = j + burn_y[k]
                if nx < 0 or ny < 0 or nx >= n or ny >= n:
                    continue
                if forest_arr[nx][ny].get_state() == 'fire':
                    forest_arr[nx][ny].set_state('burnt')
                    fire_list.remove((nx,ny))
        else:
            # Set nearby cells on fire
            for k in range(4):
                nx = i + dir_x[k]
                ny = j + dir_y[k]
                if nx < 0 or ny < 0 or nx >= n or ny >= n:
                    continue
                if forest_arr[nx][ny].get_state() == 'tree':
                    forest_arr[nx][ny].set_state('fire')
                    fire_list.append((nx, ny)) 
                

def update_fire_list():
    temp_fire_list = []
    for i in range(len(fire_list)):
        (i,j) = fire_list[i]
        # Check content of fire_list to see if it's still on fire
        if forest_arr[i][j].get_state() == 'fire':
            # Put it into new list
            if (i, j) not in temp_fire_list:
                temp_fire_list.append((i, j))
            # Search nearby cells to see if they caught on fire
            for k in range(8):
                nx = i + dir_x[k]
                ny = j + dir_y[k]
                if nx < 0 or ny < 0 or nx >= n or ny >= n:
                    continue
                # If not in list, add
                if (nx, ny) not in temp_fire_list and forest_arr[i][j].get_state() == 'fire':
                    temp_fire_list.append((nx, ny))            
                        
    return temp_fire_list

In [3]:
class trees:
    def __init__(self, position):
        self.state = 'tree'
        self.cell = box(pos=position, color=color.green*0.7)
        
    def get_state(self):
        return self.state
    
    def get_cell(self):
        return self.cell
    
    def set_state(self, s):
        self.state = s
        if self.state == 'tree':
            self.set_color(color.green*0.7)
        elif self.state == 'fire':
            self.set_color(color.red*0.7)
        elif self.state == 'burnt':
            self.set_color(color.gray(0.3))
        
    def set_color(self, color):
        self.cell.color = color

In [4]:
scene = canvas(background = color.white)

###################
n = 9                   # Set nxn matrix
start_point = n // 2    # Fire starts from the mid-bottom, so find the middle point
###################

forest_arr = [[trees(vector(i,j,0)) for j in range(n)] for i in range(n)]
forest_arr[start_point][0].set_state('fire')

# Since we do not want to iterate through the matrix to find the fire everytime, put them in fire list
fire_list = [(start_point, 0)]

time = 0
while 1:
    rate(1)
    # if everything is burned down to ashes, stop
    if (len(fire_list)) == 0:
        break
    idx = random.randrange(len(fire_list))       # Pick random cell's idx from watch_list's size
    update_forest_arr_async(n, fire_list[idx])   # Give random cell to update
    fire_list = update_fire_list()
    time += 1

print(f"total time taken: {time} seconds")

#####################
# DARK GREEN: tree state
# DARK RED: burning state
# DARK GRAY: burnt state
#
# Wanted to use texture w/ photos, but it has a lot of unknown errors, so decided to use built-in colors instead 
######################

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 